# FINAL MODEL

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score

Import the datasets

In [2]:
train = pd.read_csv('training_processed.csv')
X_train = train.drop(['ACTIVE'], axis=1) 
y_train = train['ACTIVE']

val = pd.read_csv('validation_processed.csv')
X_val = val.drop(['ACTIVE'], axis=1) 
y_val = val['ACTIVE']

total = pd.read_csv('total_processed.csv')
X_total = total.drop(['ACTIVE'], axis=1) 
y_total = total['ACTIVE']

X_test = pd.read_csv('test_smiles.csv')

Double-check the parameter settings on the validation set

In [3]:
ann = MLPClassifier(hidden_layer_sizes=(150,150,150), max_iter=100, alpha=0.001,activation='relu',solver='adam',random_state=1,learning_rate='adaptive')
ann.fit(X_train, y_train)
ann_labels = ann.predict(X_val)# predicted labels
ann_score = ann.score(X_val, y_val)
ann_pred = ann.predict_proba(X_val)

ann_auc = roc_auc_score(y_val, ann_pred[:,1])

# print the results
print('ANN Accuracy: ', ann_score)
print('ANN AUC: ', ann_auc)

Train the ANN on the full dataset and predict the test labels

In [ ]:
ann = MLPClassifier(hidden_layer_sizes=(150,150,150), max_iter=100, alpha=0.001,activation='relu',solver='adam',random_state=1,learning_rate='adaptive')
ann.fit(X_total, y_total)
ann_labels = ann.predict(X_test)# predicted labels

print(ann_labels)